# Sparse selection methods for ICTUS dataset

We implement kernelised Orthogonal Matching Pursuit and apply it to the ICTUS dataset.

Towards the end of the dataset we also evaluate performance of **LASSO** on the same dataset.

Results are not particularly promising

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib notebook

In [2]:
import sys
sys.path.append("../PyFalkon/src")
import time
import math

import numpy as np
import pandas as pd
import scipy
from scipy.spatial import distance
import matplotlib.pyplot as plt
from matplotlib import cm

from sklearn import model_selection, preprocessing
from sklearn.model_selection import train_test_split, KFold
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from falkon import Falkon 
from nystrom import select_uniform_centers
from kernels import *
from utils import load_mat_data

In [3]:
fname = "./run_all.mat"
X, Y = load_mat_data(fname)
Y[Y == 0.0] = -1.0

## Train/Test Split

In [4]:
kf = KFold(n_splits=5)
scaler = preprocessing.StandardScaler(copy=False, with_mean=True, with_std=True)

## Matching Pursuit

Decent paper documenting the algorithm:
http://www-labs.iro.umontreal.ca/~pift6080/H09/documents/papers/sparse/vincent_kernel_matching.pdf

In [5]:
def matching_pursuit(Dtr, Ytr, Dtr_ts, Yts, max_comp):
    alpha = np.zeros(Dtr.shape[1])
    res = Ytr.copy()
    Dtr_norm = np.einsum('ij,ij->j', Dtr, Dtr)
    for c in range(max_comp):
        dot = np.dot(Dtr.T, res) / Dtr_norm
        gamma_c = np.argmax(np.abs(dot))
        alpha[gamma_c] += dot[gamma_c]
        res -= alpha[gamma_c] * Dtr[:,gamma_c]
    
    # Predict
    pred_tr = np.dot(Dtr, alpha)
    print(f"Train accuracy: {np.mean(np.sign(pred_tr) == Ytr):.3f}")
    pred_ts = np.dot(Dtr_ts, alpha)
    print(f"Test accuracy: {np.mean(np.sign(pred_ts) == Yts):.3f}")

    return alpha

In [6]:
def omp(Dtr, Ytr, Dtr_ts, Yts, max_comp):
    alpha = np.zeros(Dtr.shape[1])
    res = Ytr.copy()#.reshape(-1, 1)
    Dtr_norm = np.einsum('ij,ij->j', Dtr, Dtr)
    indices = []
    for c in range(max_comp):
        dot = np.dot(Dtr.T, res) / Dtr_norm
        indices.append(np.argmax(np.abs(dot)))
        a_indices = np.asarray(indices)
        coef_upd, _, _, _ = np.linalg.lstsq(Dtr[:,a_indices], Ytr, rcond=None)
        alpha[a_indices] = coef_upd
        res = Ytr - np.dot(Dtr[:,a_indices], coef_upd)
    
    # Predict
    pred_tr = np.dot(alpha, Dtr)
    pred_ts = np.dot(alpha, Dtr_ts)
    return pred_tr, pred_ts

In [11]:
def omp_predict(num_components, kernel):
    """Run cross-validated OMP with specified parameters."""
    train_err, test_err = [], []
    for train, test in kf.split(X):
        X_train, X_test, Y_train, Y_test = X[train], X[test], Y[train], Y[test]
        scaler.fit(X_train, X_test)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

        Ktr = kernel(X_train)
        Ktr_ts = kernel(X_train, X_test)

        train_pred, test_pred = omp(Ktr, Y_train, Ktr_ts, Y_test, num_components)

        train_err.append(np.mean(np.sign(train_pred) != Y_train))
        test_err.append(np.mean(np.sign(test_pred) != Y_test))
    print("Train error: %.2f%% - Test error: %.2f%%" % 
      (np.mean(train_err)*100, np.mean(test_err)*100))

In [8]:
kernel = GaussianKernel(sigma=18, cache_enable=False)

In [9]:
omp_predict(4, kernel)

Train error: 37.56% - Test error: 37.55%


In [12]:
omp_predict(8, kernel)

Train error: 36.63% - Test error: 36.60%


In [13]:
omp_predict(20, kernel)

Train error: 35.24% - Test error: 35.26%


In [14]:
omp_predict(200, kernel)

Train error: 31.38% - Test error: 31.85%


## LASSO
We perform a couple of experiments decreasing alpha (decreasing sparsity).

In [16]:
from sklearn.linear_model import Lasso
def lasso_err(lasso):
    train_err, test_err = [], []
    for train, test in kf.split(X):
        X_train, X_test, Y_train, Y_test = X[train], X[test], Y[train], Y[test]
        scaler.fit(X_train, X_test)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

        lasso.fit(X_train, Y_train)
        pred_test = lasso.predict(X_test)
        pred_train = lasso.predict(X_train)
        train_err.append(np.mean(np.sign(pred_train) != Y_train))
        test_err.append(np.mean(np.sign(pred_test) != Y_test))
    print("Train error: %.2f%% - Test error: %.2f%%" % 
      (np.mean(train_err)*100, np.mean(test_err)*100))

In [42]:
lasso = Lasso(alpha=1, selection="random", tol=1e-3, max_iter=5_000, fit_intercept=False)
lasso_err(lasso)

Train error: 100.00% - Test error: 100.00%


In [43]:
lasso = Lasso(alpha=0.1, selection="random", tol=1e-3, max_iter=5_000, fit_intercept=False)
lasso_err(lasso)

Train error: 34.72% - Test error: 34.85%


In [44]:
lasso = Lasso(alpha=0.01, selection="random", tol=1e-3, max_iter=5_000, fit_intercept=False)
lasso_err(lasso)

Train error: 29.83% - Test error: 30.18%


In [17]:
lasso = Lasso(alpha=0.001, selection="random", tol=1e-3, max_iter=5_000, fit_intercept=False)
lasso_err(lasso)

Train error: 27.37% - Test error: 27.93%


In [18]:
lasso = Lasso(alpha=0.0001, selection="random", tol=1e-3, max_iter=5_000, fit_intercept=False)
lasso_err(lasso)

/home/giacomo/miniconda3/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 761.1928239058325, tolerance: 23.636
  positive)
/home/giacomo/miniconda3/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 741.5704797987964, tolerance: 23.636
  positive)
/home/giacomo/miniconda3/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 840.7768766783502, tolerance: 23.636
  positive)
/home/giacomo/miniconda3/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to incre

Train error: 25.94% - Test error: 27.00%


/home/giacomo/miniconda3/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 818.1667194256152, tolerance: 23.636
  positive)
